In [1]:
# __set_name__ is called first time when attr is being set, can be used to check attr name from instance


class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f"__set_name__ - {self},  {owner_class}, {property_name}")


In [2]:
class Person:
    name = ValidString()  # fired during compile time, so right away. Not when Person instance is requested

__set_name__ - <__main__.ValidString object at 0x112fe7170>,  <class '__main__.Person'>, name


In [3]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f"__set_name__ - {self},  {owner_class}, {property_name}")
        self.property_name = property_name

    def __get__(self, instance, owner_class):
        if instance is None:
            return self

        print(f"__get__ called for property `{self.property_name}` of instance `{instance}`")


class Person:
    first_name = ValidString()
    last_name = ValidString()


__set_name__ - <__main__.ValidString object at 0x112fe6b10>,  <class '__main__.Person'>, first_name
__set_name__ - <__main__.ValidString object at 0x112fe7d70>,  <class '__main__.Person'>, last_name


In [4]:
Person.first_name.property_name, Person.last_name.property_name

('first_name', 'last_name')

In [5]:
p = Person()

p.first_name

__get__ called for property `first_name` of instance `<__main__.Person object at 0x112fe4ce0>`


In [6]:
class ValidString:
    """
        Data descriptor that uses instance __dict__ to get and set values.
        It won't work if __slots__ are defined for the instance.
        (Unless __dict__ has been passed as one of the __slots__ item)
    """

    def __init__(self, min_length = None):
        self.min_length = min_length

    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError(f"`{type(instance).__name__}.{self.property_name}` should be a string.")

        if self.min_length and len(value) < self.min_length:
            raise ValueError(f"`{type(instance).__name__}.{self.property_name}` should be at least {self.min_length} characters long.")

        setattr(instance, self._property_name_key, value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self

        return getattr(instance, self._property_name_key, None)

    @property
    def _property_name_key(self):
        return f"_{self.property_name}"


class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)


In [7]:
p = Person()
p.first_name = "Alex"
try:
    p.last_name = "G"
except ValueError as e:
    print(e)


`Person.last_name` should be at least 2 characters long.


In [8]:
try:
    p.last_name = 123
except TypeError as e:
    print(e)

`Person.last_name` should be a string.


In [9]:
p.last_name = "Smith"

In [10]:
p.first_name, p.last_name

('Alex', 'Smith')

In [11]:
p.__dict__

{'_first_name': 'Alex', '_last_name': 'Smith'}

In [12]:
p = Person()
p._first_name = "Some required data"
p.__dict__

{'_first_name': 'Some required data'}

In [13]:
p.first_name = "Bob"
p.__dict__  # previous data got overwritten

{'_first_name': 'Bob'}

In [14]:
class ValidString:
    def __init__(self, min_length = None):
        self.min_length = min_length

    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError(f"`{type(instance).__name__}.{self.property_name}` should be a string.")

        if self.min_length and len(value) < self.min_length:
            raise ValueError(f"`{type(instance).__name__}.{self.property_name}` should be at least {self.min_length} characters long.")

        
        # setattr(instance, self._property_name_key, value)
        # can't use setattr as it will call __set__ and create infitite loop
        # use instance.__dict__ instead
        print("__set__ called")
        instance.__dict__[self.property_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self

        # return getattr(instance, self._property_name_key, None)
        # getattr can't be used because it causes infinite loop
        print("__get__ called")
        return instance.__dict__.get(self.property_name)


class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)


In [15]:
p = Person()
p.first_name = "Bob"

__set__ called


In [16]:
p.__dict__

{'first_name': 'Bob'}

In [17]:
try:
    p.first_name = ""  # data descriptor still works, hasn't been overwritten by the instance attribute
except ValueError as e:
    print(e)

`Person.first_name` should be at least 1 characters long.


In [18]:
p.first_name, p.last_name  # descriptor is still used!

__get__ called
__get__ called


('Bob', None)

In [19]:
p.__dict__

{'first_name': 'Bob'}